In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import src.config as cfg
import pickle
from sklearn.preprocessing import LabelEncoder

### Preprocessing

In [2]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког']

In [3]:
train = pd.read_csv(str(os.getcwd() + "/../data/raw/train.csv"))
test = pd.read_csv(str(os.getcwd() + "/../data/raw/test.csv"))

In [4]:
train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]

In [5]:
print(train.shape)
print(target.shape)

(955, 34)
(955, 5)


In [6]:
def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    return df


def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
    most_freq = df[cfg.SEX_COL].value_counts().index[0]
    df[cfg.SEX_COL] = df[cfg.SEX_COL].fillna(most_freq)
    return df


def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[cfg.CAT_COLS] = df[cfg.CAT_COLS].astype('object')

    ohe_int_cols = train[cfg.OHE_COLS].select_dtypes('number').columns
    df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

    df[cfg.REAL_COLS] = df[cfg.REAL_COLS].astype(np.float32)
    return df


def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, cfg.ID_COL)
    df = drop_unnecesary_id(df)
    df = fill_sex(df)
    df = cast_types(df)
    # df = ohe(df)
    return df

def preprocess_target(df: pd.DataFrame) -> pd.DataFrame:
    df[cfg.TARGET_COLS] = df[cfg.TARGET_COLS].astype(np.int8)
    return df

In [7]:
# preprocess_data(train).to_csv('../data/processed/train_data.csv')
# preprocess_data(test).to_csv('../data/processed/test_data.csv')

pickle.dump(preprocess_data(train), open('../data/processed/train_data.pkl', 'wb'))
pickle.dump(preprocess_target(target), open('../data/processed/target_data.pkl', 'wb'))

### Model

In [8]:
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *

In [9]:
from functools import partial

В качестве оcновной метрики оценки качества будем использовать **recall**.

Объясняется это тем, что такая метрика показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.

Нам очень важно предсказать редкие болезни у максимального количества людей, имеющих такие заблолениния, так как ошибка (то есть утверждение, что человек не болен) в данном случае будет иметь более серьезные последствия, чем если предсказать, что здоровый человек болен. А знание о, к примеру, количестве правильно предсказанных ответов по всему множеству данных (accuracy) или количестве правильно предсаказанных положительных ответов (precision) не сможет сигнализировать о неточности работы модели в рамках данной задачи.

В качестве дополнительного контроля возьмем еще метрику f1-score.

In [10]:
def scoring_recall(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return recall_score(y_true, y_pred, average='micro', zero_division=0)

def scoring_f1(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return f1_score(y_true, y_pred, average='micro')

In [11]:
train_data = pd.read_pickle("../data/processed/train_data.pkl")
target_data = pd.read_pickle("../data/processed/target_data.pkl")

In [12]:
print(train_data.shape)
print(target_data.shape)

(955, 32)
(955, 5)


In [13]:
train_data[cfg.CAT_COLS] = train_data[cfg.CAT_COLS].astype('object')

train_x, test_x, train_y, test_y = train_test_split(train_data, target_data, train_size=0.8, random_state=7) 

In [14]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, cfg.REAL_COLS),
    ('cat_cols', cat_pipe, cfg.CAT_COLS),
    ('ohe_cols', 'passthrough', cfg.OHE_COLS)
]
)

In [15]:
from catboost import CatBoostClassifier, metrics, Pool

In [26]:
model = CatBoostClassifier(
    iterations=1000,
    loss_function='MultiLogloss', eval_metric='MultiLogloss',
    learning_rate=0.03, bootstrap_type='Bayesian', boost_from_average=False,
    leaf_estimation_iterations=1, leaf_estimation_method='Gradient'
)

train_x_copy = train_x.copy()
my_pipe = preprocess_pipe.fit(train_x_copy)

train_x_copy = my_pipe.transform(train_x_copy)
# print(train_x.info())
model.fit(train_x_copy, train_y)

0:	learn: 0.6891353	total: 147ms	remaining: 2m 26s
1:	learn: 0.6849121	total: 280ms	remaining: 2m 19s
2:	learn: 0.6810042	total: 394ms	remaining: 2m 11s
3:	learn: 0.6768550	total: 430ms	remaining: 1m 47s
4:	learn: 0.6731167	total: 500ms	remaining: 1m 39s
5:	learn: 0.6694188	total: 523ms	remaining: 1m 26s
6:	learn: 0.6654680	total: 574ms	remaining: 1m 21s
7:	learn: 0.6616613	total: 687ms	remaining: 1m 25s
8:	learn: 0.6581713	total: 719ms	remaining: 1m 19s
9:	learn: 0.6546415	total: 748ms	remaining: 1m 14s
10:	learn: 0.6509454	total: 778ms	remaining: 1m 9s
11:	learn: 0.6474126	total: 827ms	remaining: 1m 8s
12:	learn: 0.6440126	total: 870ms	remaining: 1m 6s
13:	learn: 0.6405803	total: 931ms	remaining: 1m 5s
14:	learn: 0.6372909	total: 943ms	remaining: 1m 1s
15:	learn: 0.6339990	total: 1.06s	remaining: 1m 5s
16:	learn: 0.6306035	total: 1.1s	remaining: 1m 3s
17:	learn: 0.6271822	total: 1.23s	remaining: 1m 7s
18:	learn: 0.6237951	total: 1.26s	remaining: 1m 5s
19:	learn: 0.6204641	total: 1.29

In [25]:
print(train_x.shape)
print(train_x_copy.shape)
print(test_x_copy.shape)
print(test_x.shape)

(764, 32)
(764, 132)
(191, 95)
(191, 32)


In [ ]:
model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', model)
]
)

In [27]:
test_x[cfg.REAL_COLS] = test_x[cfg.REAL_COLS].astype(str)
test_x_copy = test_x.copy()
test_x_copy = my_pipe.transform(test_x_copy)
y_pred = np.array(model.predict(test_x_copy))

In [32]:
print(f'Recall score: {recall_score(test_y, y_pred, average=None)}')
print(f'F1 score: {f1_score(test_y, y_pred, average=None)}')


Recall score: [0.7173913 0.        0.        0.        0.       ]
F1 score: [0.6875 0.     0.     0.     0.    ]
